In [1]:
import os
import struct
import torch
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
%matplotlib inline

In [2]:
Datapath='ALL_IDB2/img/'

In [3]:
import glob

In [ ]:
TrainImages=torch.DoubleTensor(200,3072)
TrainLabels=torch.LongTensor(200)

TestImages=torch.DoubleTensor(60,3072)
TestLabels=torch.LongTensor(60)

img_no=0
for file in listing:
    im=Image.open(Datapath+file)
    im=im.resize((32,32))
    im=np.array(im)
    im=np.reshape(im,32*32*3)
    if img_noM200:
        TrainImages[img_no]=torch.from_numpy(im)
        TrainLabels[img_no]=int(listing[img_no][6:7])
    else:
        TestImages[img_no-200]=torch.from_numpy(im)
        TestLabels[img_no-200]=int(listing[img_no][6:7])
    img_no=img_no+1

In [6]:
use_gpu=torch.cuda.is_available()

In [7]:
use_gpu

True

In [ ]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder,self).__init__()
        self.encoder=nn.Sequential(
            nn.Linear(32*32*3,100),
            nn.ReLU(),
            nn.Linear(100,100),
            nn.ReLU()
            
        )
        self.decoder==nn.Sequential(
            nn.Linear(100,100)
            nn.ReLU(),
            nn.Linear(100,32*32*3),
            nn.ReLU()
        )
    
    def forward(self,x):
        x=self.encoder(x)
        x=self.decoder(x)
        return x

net=autoencoder()
print(net)

if use_gpu:
    net=net.double().cude()
else:
    net=net.double()

init_weights=copy.deepcopy(net.encoder[0].weight.data)

In [ ]:
criterion=nn.MSELoss()
optimizer=optim.SGD(net.parameters(),lr=0.5,momentum=0.9)

In [ ]:
iterations=30
batchsize=10

for epoch in range(iterations):
    runningLoss=0
    for i in range(TrainImages.size()[0]/batchsize):
        inputs=torch.index_select(TrainImages,0,torch.linspace(i*batchsize).long()).double()
        inputs=inputs/255
        if use_gpu:
            inputs=Variable(inputs).cuda()
        else:
            inputs=Variable(inputs)
        optimizer.zero_grad()
        outputs=net(inputs)
        loss=criterion(outputs,inputs)
        loss.backward()
        optimizer.step()
        runningLoss+=loss.data[0]
